<div style="text-align: center; background-color: #4682B4; padding: 40px; border-radius: 5px;">
    <h1> Parallelizing and Optimizing Artificial Neural Networks for Fashion-MNIST using CUDA </h1>
</div>




#### Group: 13

| **#** | **Student ID** | **Name**             |
|-------|----------------|----------------------|
| 1     | 21127159       | Nguyễn Cao Sơn       |
| 2     | 21127606       | Bùi Nguyên Hanh      |
| 3     | 21127329       | Châu Tấn Kiệt        |

# <u> **Introduction**</u>

The goal of this project is to implement and optimize an Artificial Neural Network (ANN) for the Fashion-MNIST dataset using CUDA for parallel computations. The project aims to demonstrate the efficiency of GPU-based computations through various optimization techniques and evaluate the performance improvements over baseline implementations.

#### **Project Objectives**
1. **Implementation**: Develop a fully functional ANN with:
   - One input layer (784 nodes for 28x28 grayscale images).
   - Two hidden layers (128 nodes each, with ReLU activation).
   - One output layer (10 nodes for classification, with Softmax activation).
2. **Optimization**: Apply GPU optimization techniques, such as:
   - Basic convolution kernel.
   - Shared memory and kernel fusion.
   - CUDA streams to overlap computations and memory transfers.
3. **Performance Evaluation**: Measure and analyze runtime improvements and memory efficiency.


#### **Dataset Overview**
Fashion-MNIST is a widely used dataset for machine learning and deep learning. It serves as a direct replacement for the classic MNIST dataset but introduces more complex visual patterns to represent 10 fashion categories. The dataset consists of:
- **Training Data**: 60,000 grayscale images, each labeled with one of 10 classes.
- **Test Data**: 10,000 grayscale images.
- **Image Details**:
  - Size: 28x28 pixels.
  - Format: Grayscale, normalized to values between 0 and 1.

**Classes**:

| **Category**    | **Category**     |
|------------------|------------------|
| T-shirt/top      | Sandal           |
| Trouser          | Shirt            |
| Pullover         | Sneaker          |
| Dress            | Bag              |
| Coat             | Ankle boot       |


Fashion-MNIST is challenging compared to MNIST due to the higher similarity between classes (e.g., T-shirt and Shirt), making it an excellent benchmark for testing ANN performance.

---


# <u> **1. Application Description** </u>

#### **What is your chosen application?**
The chosen application is the implementation and optimization of an Artificial Neural Network (ANN) to classify images from the Fashion-MNIST dataset using CUDA for GPU acceleration. This project aims to demonstrate how parallel computing techniques can significantly enhance the efficiency of deep learning models.

- **Input**:  
  Each input is a grayscale image of size $ 28 \times 28 $ pixels (flattened into a `784-dimensional vector`). The dataset includes `60,000 training images` and `10,000 test images`, representing `10 different classes` of fashion items.

- **Output**:  
  The output is a probability distribution over the 10 fashion categories, such as T-shirt/top, trouser, or ankle boot. The model predicts the class with the highest probability.

- **Use Cases**:  
  - Classification of fashion products in e-commerce platforms.  
  - Assisting inventory systems to automatically recognize and categorize apparel.  
  - Building real-time recommender systems for fashion items.


#### **Does it need to speed up?**
Yes, speeding up the application is critical due to the following reasons:
1. **Computational Intensity**:  
   The training of an ANN involves complex matrix operations (forward propagation, backpropagation) that scale with the size of the dataset and network layers. Running these operations sequentially on a CPU is time-consuming.
   
2. **Large Dataset**:  
   Fashion-MNIST consists of tens of thousands of high-dimensional data points, requiring efficient processing to complete training in a reasonable timeframe.

3. **Use of GPU Parallelism**:  
   CUDA enables the parallel execution of tasks such as matrix multiplications, activation functions, and gradient calculations, significantly reducing training time.

This project highlights the use of **sequential implementation** as a baseline and **parallelized GPU-based optimization** to achieve improved performance. The outcomes will show how leveraging parallelism impacts real-world applications in AI and machine learning.


# <u> **2. Sequential Implementation**  </u>

#### **Design**
The sequential implementation involves executing the entire neural network training process step-by-step on the CPU, from input to output.


**Step 1: Dataset Loading**
- **Input**:
  - Fashion-MNIST dataset files: `train-images-idx3-ubyte` and `train-labels-idx1-ubyte`.
- **Process**:
  1. Open the binary files containing the image data and labels.
  2. Extract the metadata:
     - Number of images.
     - Image dimensions (28x28 pixels).
  3. Normalize pixel values from $[0, 255]$ to $[0, 1]$ for better numerical stability during training.
  4. Store images as flattened vectors of size $784$ and labels as integers.
- **Output**:
  - A list of normalized image vectors.
  - A corresponding list of integer labels.


**Step 2: Model Initialization**
- **Input**:
  - Neural network architecture details:
    - Input layer: $784$ nodes.
    - Two hidden layers: $128$ nodes each (with ReLU activation).
    - Output layer: $10$ nodes (with Softmax activation).
- **Process**:
  1. Initialize weight matrices ($W1, W2, W3$) and bias vectors ($b1, b2, b3$) for each layer.
  2. Use **Xavier initialization** to set weights to small random values:
     $$
     W \sim \mathcal{N}\left(0, \sqrt{\frac{2}{\text{in\_nodes} + \text{out\_nodes}}}\right)
     $$
  3. Set biases to zeros.
- **Output**:
  - Initialized weights and biases ready for training.


**Step 3: Forward Propagation**
- **Input**:
  - A single normalized image vector (size $784$).
  - Initialized weights and biases.
- **Process**:
  1. **Layer 1 (Input to Hidden Layer 1)**:
     - Compute $Z1 = W1 \cdot X + b1$, where:
       - $Z1$ is the pre-activation output.
       - $X$ is the input vector.
     - Apply ReLU activation:
       $$
       A1 = \text{ReLU}(Z1) = \max(0, Z1)
       $$
  2. **Layer 2 (Hidden Layer 1 to Hidden Layer 2)**:
     - Compute $Z2 = W2 \cdot A1 + b2$.
     - Apply ReLU activation:
       $$
       A2 = \text{ReLU}(Z2) = \max(0, Z2)
       $$
  3. **Output Layer (Hidden Layer 2 to Output)**:
     - Compute $Z3 = W3 \cdot A2 + b3$.
     - Apply Softmax activation:
       $$
       \text{Softmax}(Z3_i) = \frac{\exp(Z3_i)}{\sum_j \exp(Z3_j)}
       $$
- **Output**:
  - The final probability distribution over the $10$ output classes.


**Step 4: Loss Calculation**
- **Input**:
  - Model output probabilities from the Softmax function.
  - True label (one-hot encoded vector).
- **Process**:
  - Compute the **Cross-Entropy Loss**:
    $$
    L = -\sum_{i=1}^{10} y_i \cdot \log(\hat{y}_i)
    $$
    Where:
    - $y_i$ is the true label (one-hot encoded).
    - $\hat{y}_i$ is the predicted probability for class $i$.
- **Output**:
  - Scalar loss value representing the model's error.


**Step 5: Backward Propagation**
- **Input**:
  - Output probabilities, true label, and intermediate activations ($A1, A2$).
- **Process**:
  1. **Output Layer Gradients**:
     - Compute:
       $$
       \delta_3 = \hat{y} - y
       $$
       $$
       \frac{\partial L}{\partial W3} = \delta_3 \cdot A2^T
       $$
       $$
       \frac{\partial L}{\partial b3} = \delta_3
       $$
  2. **Hidden Layer 2 Gradients**:
     - Backpropagate the error:
       $$
       \delta_2 = (W3^T \cdot \delta_3) \circ \text{ReLU}'(Z2)
       $$
       $$
       \frac{\partial L}{\partial W2} = \delta_2 \cdot A1^T
       $$
       $$
       \frac{\partial L}{\partial b2} = \delta_2
       $$
  3. **Hidden Layer 1 Gradients**:
     - Backpropagate further:
       $$
       \delta_1 = (W2^T \cdot \delta_2) \circ \text{ReLU}'(Z1)
       $$
       $$
       \frac{\partial L}{\partial W1} = \delta_1 \cdot X^T
       $$
       $$
       \frac{\partial L}{\partial b1} = \delta_1
       $$
- **Output**:
  - Gradients for all weights and biases.


**Step 6: Parameter Update**
- **Input**:
  - Gradients ($\frac{\partial L}{\partial W}, \frac{\partial L}{\partial b}$).
  - Learning rate ($\eta$).
- **Process**:
  - Update weights and biases:
    $$
    W = W - \eta \cdot \frac{\partial L}{\partial W}
    $$
    $$
    b = b - \eta \cdot \frac{\partial L}{\partial b}
    $$
- **Output**:
  - Updated weights and biases.


**Step 7: Training Loop**
- **Process**:
  1. Repeat steps 3-6 for all training samples in an epoch.
  2. Track metrics such as:
     - Average loss per epoch.
     - Accuracy on training data.
  3. Continue for a fixed number of epochs or until convergence.
- **Output**:
  - Final trained model parameters.


#### **Evaluate**

1. **Experiment Setup**:
   - Run on a CPU using the Eigen library for linear algebra.
      - Try on PC and Colab
   - Use the entire Fashion-MNIST dataset for training.
      - `train-images-idx3-ubyte`
      - `train-labels-idx1-ubyte`

2. **Execution**:
   - Observe the training process over multiple epochs (about 10 epochs).

3. **Correctness**:
   - Check that the loss decreases over time.
   - Verify the model's predictions against ground truth labels to confirm accuracy.

![{FD6AE7AA-2EA0-4961-8516-DE6DCFE84B28}.png](attachment:{FD6AE7AA-2EA0-4961-8516-DE6DCFE84B28}.png)



# <u> Parallel Implementation </u>

#### **Parallelized Steps**

1. **Forward Propagation**  
   a) **Matrix Multiplication Operations (`matrix_multiply_kernel`)**  
   **Reason**:  
   - Highly parallelizable operation.  
   - Computationally intensive with  $ O(M \times N \times K) $  complexity.  
   - Each output element can be computed independently.  
   - Perfect for GPU's SIMD (Single Instruction, Multiple Data) architecture.  

   b) **ReLU Activation (`relu_kernel`)**  
   **Reason**:  
   - Element-wise operation.  
   - Each neuron's activation can be computed independently.  
   - No dependencies between elements.  
   - Easily parallelizable across multiple threads.  

   c) **Softmax Computation (`softmax_kernel`)**  
   - Requires two passes: max finding and exponential normalization.  
   - While there are dependencies (e.g., needs the sum of all elements), it can still benefit from parallel reduction.  
   - Critical for classification output.

2. **Backward Propagation**  
   **Complete Backward Pass (`backward_propagation_kernel`)**  
   **Reason**:  
   - Gradient computations involve matrix operations.  
   - Weight updates can be done in parallel.  
   - Multiple independent calculations are performed for each layer.  

---


# <u> **4. Parallel Implementation + Optimization** </u>




## **1. Optimization 1: Backward Propagation Kernel**

The key optimization in this phase focuses on improving the backward propagation kernel for better performance and efficiency.



#### **What is Optimized**
- **Weight and Bias Updates**: Improved efficiency in updating weights and biases during backpropagation.
- **Memory Access Patterns**: Optimized for faster and more efficient memory operations.
- **Thread Synchronization**: Minimized overhead from thread synchronization to boost performance.



#### **Why This Part**
- **Computational Intensity**: Backward propagation is one of the most computationally expensive processes in training.
- **Race Conditions**: Involves numerous parallel weight updates, which can lead to race conditions without proper handling.
- **Memory Conflicts**: High potential for memory conflicts and synchronization overhead, requiring optimization.



#### **Optimization Details**

a) **Atomic Operations**:
- Utilizes CUDA atomic operations to prevent race conditions.
- Ensures thread-safe updates to weights and biases.
- Eliminates the need for multiple synchronization barriers.

b) **Grid/Block Configuration**:
- Employs more efficient thread block organization.
- Uses a single unified grid dimension for all layers.
- Achieves better load balancing across streaming multiprocessors (SMs).

c) **Memory Access Pattern**:
- Improves shared memory usage for faster data access.
- Optimizes memory coalescing to reduce latency.
- Minimizes bank conflicts for smoother memory operations.

d) **Synchronization Optimization**:
- Reduces the number of `__syncthreads()` calls.
- Places synchronization barriers more efficiently.
- Implements better thread cooperation patterns to minimize idle time.


#### **Other Benefits**

1. **Forward Propagation**:
   - The structure remains unchanged but benefits from:
   - Improved grid/block configuration.
   - Enhanced memory access patterns via better thread organization.

2. **Memory Management**:
   - Implements a more efficient allocation and deallocation pattern.
   - Reuses allocated memory to minimize overhead.
   - Reduces memory transfer operations between host and device.

---


## **Optimization 2: Utilizing CUDA Streams for Asynchronous Execution**

#### **Analyze**
CUDA streams are used to enable asynchronous execution of tasks, allowing overlapping of computation and data transfers. This approach improves overall GPU utilization by minimizing idle time and increasing throughput.


#### **Key Idea**
- CUDA streams allow the GPU to execute multiple tasks concurrently by using separate execution queues.
- Asynchronous execution enables overlapping tasks such as computation and memory transfers, reducing latency and improving performance.

**Examples of Overlapping Tasks**:
1. Forward propagation runs while data for the next batch is transferred to the GPU.
2. Backward propagation executes while the output of forward propagation is copied back to the host.


#### **Key Improvements**

1. **Using `cudaMemcpyAsync` for Asynchronous Data Transfers**:
   - Input and target data are transferred asynchronously using separate streams:
     - `stream1` for input data.
     - `stream2` for target labels.
   - Reduces the latency caused by waiting for data transfers to complete before starting computations.

2. **Separate Streams for Kernels**:
   - **Stream3**: Executes the `forward_propagation_kernel`.
   - **Stream4**: Executes the `backward_propagation_kernel`.
   - Allows forward propagation to process the current batch while backward propagation processes the previous batch, enabling pipeline execution.

3. **Asynchronous Data Copying Back**:
   - The output of the forward propagation is transferred back to the host asynchronously via `stream1`, allowing backward propagation to continue without waiting for the transfer to finish.


#### **Optimized Process**
1. **Memory Allocation**:
   - Similar to the standard memory allocation, but uses pinned memory for efficient asynchronous transfers.

2. **Copying Input and Target Data**:
   - Input data is copied to the GPU asynchronously using `stream1`.
   - Target labels are copied using `stream2`.

3. **Forward Propagation**:
   - The `forward_propagation_kernel` runs in `stream3`, processing the current batch of data.

4. **Backward Propagation**:
   - The `backward_propagation_kernel` runs in `stream4`, updating weights and biases for the previous batch.

5. **Asynchronous Output Transfer**:
   - The output of the forward propagation is copied back to the host asynchronously via `stream1` while the GPU continues with backward propagation.


#### **Benefits of CUDA Streams**
- **Reduced Latency**:
  - Overlapping data transfers and computations minimizes idle time and reduces overall latency.
- **Increased Throughput**:
  - Parallel execution of tasks allows the GPU to handle more operations in the same timeframe.
- **Better GPU Utilization**:
  - Tasks across different streams make full use of available GPU resources, maximizing performance.


### **5. Reflection**

#### **Difficulties**
- Managing data transfers between CPU and GPU while optimizing memory coalescing required significant effort.
- Synchronizing team efforts and integrating GPU-optimized kernels with host code proved time-consuming.
- Limited hardware availability slowed testing and debugging processes.

#### **Learnings**
- Gained a deeper understanding of CUDA programming, including kernel fusion, shared memory optimization, and asynchronous operations.
- Learned the importance of profiling tools like `nvprof` to identify bottlenecks and improve performance.
- Improved collaboration and problem-solving skills by combining expertise in CUDA, machine learning, and mathematical modeling.

#### **What Would Be Improved or Added with More Time**
1. **Advanced Optimizations**:
   - Utilize warp-level primitives and tensor cores for better performance.
   - Implement mixed-precision training to improve speed and memory efficiency.
2. **Testing and Scalability**:
   - Conduct broader testing across different hardware and datasets.
3. **Visualization and Documentation**:
   - Add real-time training visualizations (loss and accuracy).
   - Enhance documentation and provide a detailed user guide for future improvements.

---


### **6. References**
List materials and resources consulted during the project:
  - [ANN Documentation](https://www.geeksforgeeks.org/building-artificial-neural-networks-ann-from-scratch).
  - [CUDA Documentation](https://www.cs.utexas.edu/~rossbach/cs380p/papers/cuda-programming.pdf)
  - [Eigen Library Documentation](https://eigen.tuxfamily.org/index.php?title=Main_Page#Download).
  - [Fashion-MNIST Dataset Source](https://github.com/zalandoresearch/fashion-mnist).
  - [Neural Network](http://neuralnetworksanddeeplearning.com/)
  - [Layers in ANN](https://www.geeksforgeeks.org/layers-in-artificial-neural-networks-ann/)
---

# <u> Demonstration </u>

Environment setup - Eigen library installation

In [ ]:
!wget https://gitlab.com/libeigen/eigen/-/archive/3.4.0/eigen-3.4.0.tar.gz
!tar -xvzf eigen-3.4.0.tar.gz

--2024-12-22 14:19:03--  https://gitlab.com/libeigen/eigen/-/archive/3.4.0/eigen-3.4.0.tar.gz
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘eigen-3.4.0.tar.gz’

eigen-3.4.0.tar.gz      [ <=>                ]   2.58M  --.-KB/s    in 0.02s   

2024-12-22 14:19:04 (139 MB/s) - ‘eigen-3.4.0.tar.gz’ saved [2705005]

eigen-3.4.0/
eigen-3.4.0/.gitignore
eigen-3.4.0/.gitlab-ci.yml
eigen-3.4.0/.gitlab/
eigen-3.4.0/.gitlab/issue_templates/
eigen-3.4.0/.gitlab/issue_templates/Bug Report.md
eigen-3.4.0/.gitlab/issue_templates/Feature Request.md
eigen-3.4.0/.gitlab/merge_request_templates/
eigen-3.4.0/.gitlab/merge_request_templates/Merge Request Template.md
eigen-3.4.0/.hgeol
eigen-3.4.0/CMakeLists.txt
eigen-3.4.0/COPYING.APACHE
eigen-3.4.0/COPYING.BSD
eigen-3.4.0/COPYING.GPL
eigen-3

Compile and Run - CPU

In [ ]:
!g++ fashion_mnist_host.cpp -I eigen-3.4.0 -o fashion_mnist_host

In [ ]:
!./fashion_mnist_host

Loaded 60000 training samples.
Epoch 1, Average Loss: 1.12491, Accuracy: 0.564833, Epoch Time: 512709 ms
Epoch 2, Average Loss: 0.448881, Accuracy: 0.836367, Epoch Time: 509094 ms
Epoch 3, Average Loss: 0.392269, Accuracy: 0.8567, Epoch Time: 509171 ms
Epoch 4, Average Loss: 0.361197, Accuracy: 0.8676, Epoch Time: 510404 ms
Epoch 5, Average Loss: 0.340561, Accuracy: 0.87405, Epoch Time: 512290 ms
Epoch 6, Average Loss: 0.321715, Accuracy: 0.88015, Epoch Time: 510359 ms
Epoch 7, Average Loss: 0.310083, Accuracy: 0.884433, Epoch Time: 511706 ms
Epoch 8, Average Loss: 0.301402, Accuracy: 0.8872, Epoch Time: 508566 ms
Epoch 9, Average Loss: 0.290712, Accuracy: 0.890967, Epoch Time: 512030 ms
Epoch 10, Average Loss: 0.283167, Accuracy: 0.89435, Epoch Time: 514994 ms


Basic Kernel

In [ ]:
!nvcc -o basic_kernel BasicKernel.cu


In [ ]:
!./basic_kernel


Loaded 60000 training samples.
Epoch 1, Average Loss: 1.12702, Accuracy: 0.561633, Epoch Time: 16458 ms
Epoch 2, Average Loss: 0.449983, Accuracy: 0.8366, Epoch Time: 16339 ms
Epoch 3, Average Loss: 0.391583, Accuracy: 0.856583, Epoch Time: 16473 ms
Epoch 4, Average Loss: 0.360872, Accuracy: 0.86775, Epoch Time: 16296 ms
Epoch 5, Average Loss: 0.338907, Accuracy: 0.875167, Epoch Time: 16040 ms
Epoch 6, Average Loss: 0.322671, Accuracy: 0.880433, Epoch Time: 16191 ms
Epoch 7, Average Loss: 0.309904, Accuracy: 0.884417, Epoch Time: 16073 ms
Epoch 8, Average Loss: 0.294593, Accuracy: 0.8887, Epoch Time: 16276 ms
Epoch 9, Average Loss: 0.287604, Accuracy: 0.891883, Epoch Time: 16490 ms
Epoch 10, Average Loss: 0.27525, Accuracy: 0.897617, Epoch Time: 16221 ms


Optimized Kernel 1

In [ ]:
!nvcc -o optimized_kernel OptimizedKernel1.cu

In [ ]:
!./optimized_kernel


Loaded 60000 training samples.
Epoch 1, Average Loss: 1.14724, Accuracy: 0.558883, Epoch Time: 13680 ms
Epoch 2, Average Loss: 0.452151, Accuracy: 0.833683, Epoch Time: 13449 ms
Epoch 3, Average Loss: 0.395224, Accuracy: 0.85385, Epoch Time: 13340 ms
Epoch 4, Average Loss: 0.363905, Accuracy: 0.864767, Epoch Time: 13329 ms
Epoch 5, Average Loss: 0.343728, Accuracy: 0.87155, Epoch Time: 13288 ms
Epoch 6, Average Loss: 0.327634, Accuracy: 0.877017, Epoch Time: 13278 ms
Epoch 7, Average Loss: 0.315788, Accuracy: 0.8817, Epoch Time: 13283 ms
Epoch 8, Average Loss: 0.303139, Accuracy: 0.88655, Epoch Time: 13315 ms
Epoch 9, Average Loss: 0.294405, Accuracy: 0.889617, Epoch Time: 13323 ms
Epoch 10, Average Loss: 0.285016, Accuracy: 0.8929, Epoch Time: 13303 ms


Optimized Kernel 2

In [ ]:
!nvcc -o stream_optimized StreamOptimized.cu
!./stream_optimized

Loaded 60000 training samples.
Epoch 1, Average Loss: 1.1485, Accuracy: 0.555783, Epoch Time: 16064 ms
Epoch 2, Average Loss: 0.453167, Accuracy: 0.833933, Epoch Time: 15852 ms
Epoch 3, Average Loss: 0.396645, Accuracy: 0.854833, Epoch Time: 16089 ms
Epoch 4, Average Loss: 0.365575, Accuracy: 0.865783, Epoch Time: 15911 ms
Epoch 5, Average Loss: 0.343712, Accuracy: 0.873267, Epoch Time: 15931 ms
Epoch 6, Average Loss: 0.326801, Accuracy: 0.8783, Epoch Time: 16003 ms
Epoch 7, Average Loss: 0.312534, Accuracy: 0.883133, Epoch Time: 16010 ms
Epoch 8, Average Loss: 0.300962, Accuracy: 0.887083, Epoch Time: 15894 ms
Epoch 9, Average Loss: 0.291087, Accuracy: 0.89075, Epoch Time: 15967 ms
Epoch 10, Average Loss: 0.28473, Accuracy: 0.892517, Epoch Time: 16121 ms
